## Querys em SQL

### Conectando ao DataWarehouse

In [ ]:
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#Configurando a conexão com o SQL Server (via atuenticação do Windows, pois já estou conectado como localhost)
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=DESKTOP-U623P07;'
                      'DATABASE=vendas_demanda_dw;'
                      'Trusted_Connection=yes;')

#Função para executar consultas e retornar um dataframe
def run_query(query):
    return pd.read_sql(query, conn)

### Análise Geral de Vendas

#### Vendas Totais por Período

In [23]:
query_vendas_periodo = """
SELECT 
    t.data_venda,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_tempo t ON f.data_venda = t.data_venda
GROUP BY t.data_venda
ORDER BY t.data_venda
"""

df_vendas_periodo = run_query(query_vendas_periodo)
print(df_vendas_periodo)

     data_venda  valor_total_vendas
0    2023-07-09            11945.50
1    2023-07-10            17276.72
2    2023-07-11            14191.39
3    2023-07-12            20556.06
4    2023-07-13            28896.07
..          ...                 ...
725  2025-07-03            36521.48
726  2025-07-04            11795.98
727  2025-07-05            15444.42
728  2025-07-06            23964.73
729  2025-07-07            23372.13

[730 rows x 2 columns]


#### Vendas por Categoria de Produto

In [24]:
query_vendas_categoria = """
SELECT 
    p.categoria_produto,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_produto p ON f.produto_id = p.produto_id
GROUP BY p.categoria_produto
ORDER BY valor_total_vendas DESC
"""

df_vendas_categoria = run_query(query_vendas_categoria)
print(df_vendas_categoria)

  categoria_produto  valor_total_vendas
0       Eletrônicos          1903371.42
1         Alimentos          1771121.29
2            Beleza          1763889.39
3            Livros          1731970.92
4          Esportes          1727323.61
5        Brinquedos          1587731.73
6              Casa          1556884.46
7            Roupas          1540870.71


### Análise de Clientes

#### Distribuição de Clientes por Gênero

In [25]:
query_clientes_genero = """
SELECT 
    c.genero,
    COUNT(*) AS numero_clientes
FROM dim_cliente c
GROUP BY c.genero
ORDER BY numero_clientes DESC
"""

df_clientes_genero = run_query(query_clientes_genero)
print(df_clientes_genero)

  genero  numero_clientes
0      F              184
1  Outro              169
2      M              147


#### Clientes por Faixa Etária

In [26]:
query_clientes_idade = """
SELECT 
    CASE 
        WHEN c.idade < 18 THEN 'Menor de 18'
        WHEN c.idade BETWEEN 18 AND 24 THEN '18-24'
        WHEN c.idade BETWEEN 25 AND 34 THEN '25-34'
        WHEN c.idade BETWEEN 35 AND 44 THEN '35-44'
        WHEN c.idade BETWEEN 45 AND 54 THEN '45-54'
        WHEN c.idade BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65+'
    END AS faixa_etaria,
    COUNT(*) AS numero_clientes
FROM dim_cliente c
GROUP BY 
    CASE 
        WHEN c.idade < 18 THEN 'Menor de 18'
        WHEN c.idade BETWEEN 18 AND 24 THEN '18-24'
        WHEN c.idade BETWEEN 25 AND 34 THEN '25-34'
        WHEN c.idade BETWEEN 35 AND 44 THEN '35-44'
        WHEN c.idade BETWEEN 45 AND 54 THEN '45-54'
        WHEN c.idade BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65+'
    END
ORDER BY faixa_etaria
"""

df_clientes_idade = run_query(query_clientes_idade)
print(df_clientes_idade)

  faixa_etaria  numero_clientes
0        18-24               56
1        25-34               75
2        35-44               91
3        45-54               72
4        55-64               70
5          65+              136


#### Clientes Ativos vs Inativos

In [27]:
query_clientes_ativos = """
SELECT 
    CASE 
        WHEN DATEDIFF(DAY, c.data_cadastro, GETDATE()) < 30 THEN 'Ativo'
        ELSE 'Inativo'
    END AS status_cliente,
    COUNT(*) AS numero_clientes
FROM dim_cliente c
GROUP BY 
    CASE 
        WHEN DATEDIFF(DAY, c.data_cadastro, GETDATE()) < 30 THEN 'Ativo'
        ELSE 'Inativo'
    END
"""

df_clientes_ativos = run_query(query_clientes_ativos)
print(df_clientes_ativos)

  status_cliente  numero_clientes
0          Ativo               11
1        Inativo              489


### Análise de Produtos

#### Top 10 Produtos Mais Vendidos

In [28]:
query_top_produtos = """
SELECT TOP 10
    p.nome_produto,
    SUM(f.quantidade) AS quantidade_vendida
FROM fato_vendas f
JOIN dim_produto p ON f.produto_id = p.produto_id
GROUP BY p.nome_produto
ORDER BY quantidade_vendida DESC
"""

df_top_produtos = run_query(query_top_produtos)
print(df_top_produtos)

                       nome_produto  quantidade_vendida
0              Produto Explicabo et                 258
1                  Produto Error ex                 249
2        Produto Adipisci doloribus                 248
3  Produto Explicabo necessitatibus                 239
4                     Produto Id ad                 238
5                    Produto A quam                 232
6           Produto Incidunt magnam                 232
7              Produto Dolore nulla                 226
8            Produto Sed distinctio                 226
9        Produto Perspiciatis ipsam                 225


#### Produtos por Categoria e SubCategoria

In [29]:
query_produtos_categoria = """
SELECT 
    p.categoria_produto,
    p.subcategoria_produto,
    COUNT(*) AS numero_produtos
FROM dim_produto p
GROUP BY p.categoria_produto, p.subcategoria_produto
ORDER BY p.categoria_produto, numero_produtos DESC
"""

df_produtos_categoria = run_query(query_produtos_categoria)
print(df_produtos_categoria)

   categoria_produto subcategoria_produto  numero_produtos
0          Alimentos              Bebidas                5
1          Alimentos           Congelados                5
2          Alimentos       Não Perecíveis                5
3          Alimentos           Perecíveis                5
4             Beleza            Maquiagem                5
5             Beleza  Cuidados com a Pele                5
6             Beleza          Fragrâncias                5
7             Beleza               Cabelo                5
8         Brinquedos            Carrinhos                5
9         Brinquedos              Bonecas                5
10        Brinquedos           Educativos                5
11        Brinquedos                Jogos                5
12              Casa               Jardim                5
13              Casa               Móveis                5
14              Casa           Utensílios                5
15              Casa            Decoração               

### Análise de Canais de Vendas

#### Vendas Por Canais de Vendas

In [30]:
query_vendas_canal = """
SELECT 
    l.canal_venda,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_loja l ON f.loja_id = l.loja_id
GROUP BY l.canal_venda
ORDER BY valor_total_vendas DESC
"""

df_vendas_canal = run_query(query_vendas_canal)
print(df_vendas_canal)

   canal_venda  valor_total_vendas
0  Marketplace          5521450.63
1       Online          4065936.33
2  Loja Física          3995776.57


#### Vendas Por Método de Pagamento

In [31]:
query_vendas_pagamento = """
SELECT 
    l.metodo_pagamento,
    COUNT(*) AS numero_vendas,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_loja l ON f.loja_id = l.loja_id
GROUP BY l.metodo_pagamento
ORDER BY valor_total_vendas DESC
"""

df_vendas_pagamento = run_query(query_vendas_pagamento)
print(df_vendas_pagamento)

  metodo_pagamento  numero_vendas  valor_total_vendas
0           Boleto           3996          5522591.17
1   Cartão Crédito           2945          3909999.89
2              PIX           1991          2687894.52
3    Cartão Débito           1068          1462677.95


### Análise de Sazonalidade

#### Vendas Por Dia da Semana

In [36]:
query_vendas_dia_semana = """
SELECT 
    t.dia_semana,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_tempo t ON f.data_venda = t.data_venda
GROUP BY t.dia_semana
ORDER BY 
    CASE t.dia_semana
        WHEN 'Monday' THEN 1
        WHEN 'Tuesday' THEN 2
        WHEN 'Wednesday' THEN 3
        WHEN 'Thursday' THEN 4
        WHEN 'Friday' THEN 5
        WHEN 'Saturday' THEN 6
        WHEN 'Sunday' THEN 7
    END
"""

df_vendas_dia_semana = run_query(query_vendas_dia_semana)
print(df_vendas_dia_semana)

  dia_semana  valor_total_vendas
0     Monday          1998817.09
1    Tuesday          1940542.88
2  Wednesday          1984900.57
3   Thursday          1856496.07
4     Friday          1897804.28
5   Saturday          1913577.27
6     Sunday          1991025.37


#### Vendas Por Mês

In [33]:
query_vendas_mes = """
SELECT 
    t.mes,
    SUM(f.valor_total) AS valor_total_vendas
FROM fato_vendas f
JOIN dim_tempo t ON f.data_venda = t.data_venda
GROUP BY t.mes
ORDER BY t.mes
"""

df_vendas_mes = run_query(query_vendas_mes)
print(df_vendas_mes)

    mes  valor_total_vendas
0     1          1087224.49
1     2          1098514.05
2     3          1092142.46
3     4          1115339.26
4     5          1124338.71
5     6          1165658.55
6     7          1152748.42
7     8          1166010.46
8     9          1037132.59
9    10          1181778.22
10   11          1176591.99
11   12          1185684.33


### Análise de Performance

#### Top 10 Clientes Por Valor Gasto

In [34]:
query_top_clientes = """
SELECT TOP 10
    c.cliente_id,
    c.nome,
    SUM(f.valor_total) AS valor_total_gasto
FROM fato_vendas f
JOIN dim_cliente c ON f.cliente_id = c.cliente_id
GROUP BY c.cliente_id, c.nome
ORDER BY valor_total_gasto DESC
"""

df_top_clientes = run_query(query_top_clientes)
print(df_top_clientes)

   cliente_id                     nome  valor_total_gasto
0         331        Stephany Siqueira           55401.57
1         134        Ravi Lucca Vargas           50977.45
2          95  Gustavo Henrique Farias           49771.55
3         491             Sara Ribeiro           49163.19
4         426               Noah Moura           49118.24
5         220          Mirella Sampaio           47587.61
6         483              Eloah Sousa           46373.29
7         470     Ana Vitória Cassiano           46020.92
8         101         Sra. Rebeca Rios           45961.34
9         266                Léo Silva           45813.70


#### Margem de Lucro por Categoria de Produto

In [35]:
query_margem_categoria = """
SELECT 
    p.categoria_produto,
    SUM(f.valor_total - (f.quantidade * p.custo)) AS lucro_total
FROM fato_vendas f
JOIN dim_produto p ON f.produto_id = p.produto_id
GROUP BY p.categoria_produto
ORDER BY lucro_total DESC
"""

df_margem_categoria = run_query(query_margem_categoria)
print(df_margem_categoria)

  categoria_produto  lucro_total
0            Beleza    824260.76
1         Alimentos    797222.35
2       Eletrônicos    780693.74
3        Brinquedos    773323.36
4            Livros    721656.53
5              Casa    708412.15
6          Esportes    703898.88
7            Roupas    651659.62
